In [1]:
import numpy as np

def gaussQuadStd1d(g, noOfIntegPt):
    """Computes the integral ∫ g(ξ) dξ using Gaussian quadrature on [-1,1]"""
    if noOfIntegPt == 2:
        points = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])
        weights = np.array([1, 1])
    elif noOfIntegPt == 3:
        points = np.array([-np.sqrt(3/5), 0, np.sqrt(3/5)])
        weights = np.array([5/9, 8/9, 5/9])
    else:
        return 0
    
    return sum(w * g(xi) for w, xi in zip(weights, points))

def gaussQuad1d(fn, lowerLimit, upperLimit, noOfIntegPt):
    transform = lambda ξ: (upperLimit - lowerLimit) / 2 * ξ + (upperLimit + lowerLimit) / 2
    g = lambda ξ: fn(transform(ξ)) * (upperLimit - lowerLimit) / 2 

    return gaussQuadStd1d(g, noOfIntegPt)

def shapeFn1d(i, x, x1, x2, p):
    
    if x < x1 or x > x2:
        return 0  
    
    if p == 1:  
        if i == 1:
            return (x2 - x) / (x2 - x1)
        elif i == 2:
            return (x - x1) / (x2 - x1)
    
    elif p == 2: 
        mid = (x1 + x2) / 2  
        
        if i == 1:
            return (x - mid) * (x - x2) / ((x1 - mid) * (x1 - x2))
        elif i == 2:
            return (x - x1) * (x - x2) / ((mid - x1) * (mid - x2))
        elif i == 3:
            return (x - x1) * (x - mid) / ((x2 - x1) * (x2 - mid))
    
    return 0  


def shapeFnDer1d(i, x, x1, x2, p):
    
    if x < x1 or x > x2:
        return 0  
    
    if p == 1:  
        if i == 1:
            return -1 / (x2 - x1)
        elif i == 2:
            return 1 / (x2 - x1)
    
    elif p == 2:  
        mid = (x1 + x2) / 2  
        
        if i == 1:
            return ((2*x - x2 - mid) / ((x1 - mid) * (x1 - x2)))
        elif i == 2:
            return ((2*x - x1 - x2) / ((mid - x1) * (mid - x2)))
        elif i == 3:
            return ((2*x - x1 - mid) / ((x2 - x1) * (x2 - mid)))
    
    return 0
def meij(e, i, j, xh, shapeFn, noOfIntegPt):
    """Computes the mass matrix entry M^e_ij."""
    x1, x2 = xh[e-1], xh[e]
    
    def integrand(xi):
        x = 0.5 * ((x2 - x1) * xi + x1 + x2)  # Transform ξ to x
        return shapeFn1d(i, x, x1, x2, shapeFn) * shapeFn1d(j, x, x1, x2, shapeFn) * (x2 - x1) / 2

    return gaussQuadStd1d(integrand, noOfIntegPt)

def keij(a, c, e, i, j, xh, shapeFn, noOfIntegPt):
    """Computes the stiffness matrix entry K^e_ij."""
    x1, x2 = xh[e-1], xh[e]

    def integrand(xi):
        x = 0.5 * ((x2 - x1) * xi + x1 + x2)  # Transform ξ to x
        a_val = a(x)
        c_val = c(x)
        dphi_i = shapeFnDer1d(i, x, x1, x2, shapeFn)
        dphi_j = shapeFnDer1d(j, x, x1, x2, shapeFn)
        phi_i = shapeFn1d(i, x, x1, x2, shapeFn)
        phi_j = shapeFn1d(j, x, x1, x2, shapeFn)

        return (a_val * dphi_i * dphi_j + c_val * phi_i * phi_j) * (x2 - x1) / 2

    return gaussQuadStd1d(integrand, noOfIntegPt)

def fei(a, c, f, p0, e, i, xh, shapeFn, noOfIntegPt):
    """Computes the load vector entry F^e_i."""
    x1, x2 = xh[e-1], xh[e]

    def integrand(xi):
        x = 0.5 * ((x2 - x1) * xi + x1 + x2)  # Transform ξ to x
        f_val = f(x)
        a_val = a(x)
        c_val = c(x)
        dphi_i = shapeFnDer1d(i, x, x1, x2, shapeFn)
        phi_i = shapeFn1d(i, x, x1, x2, shapeFn)
        G = p0*shapeFn1d(1,x,x1,x2,shapeFn)
        Gx = p0*shapeFnDer1d(1,x,x1,x2,shapeFn)
        return (f_val * shapeFn1d(i, x, x1, x2, shapeFn) - a_val*Gx*dphi_i - c_val*G*phi_i) * (x2 - x1) / 2

    return gaussQuadStd1d(integrand, noOfIntegPt)
    
    if e == 1 and i == 1:  # Adjust for Dirichlet boundary at x=0
        fei -= p0 * a(x1) * shapeFn1d(i, x1, x1, x2, shapeFn)

In [39]:
a = lambda x: 3*x + x**2
c = lambda x: x
f = lambda x: np.sin(x)
p0 = 2
xh = np.linspace(0.,2.,6)
print(f'meij(4,1,2,xh,1,3) = { meij(4,1,2,xh,1,3) }')
print(f'keij(a,c,4,1,2,xh,1,3) = {keij(a,c,4,1,2,xh,1,3) }')
print(f'fei(a,c,f,p0,1,2,xh,1,3) = {fei(a,c,f,p0,1,2,xh,1,3)}')

meij(4,1,2,xh,1,3) = 0.06666666666666668
keij(a,c,4,1,2,xh,1,3) = -15.340000000000003
fei(a,c,f,p0,1,2,xh,1,3) = 3.2924848499213506


In [3]:
print(f' meij(4,3,2,xh,2,3) = {meij(4,3,2,xh,2,3)}')
print(f' keij(a,c,4,1,2,xh,2,3) = {keij(a,c,4,1,2,xh,2,3) }')
print(f' fei(a,c,f,p0,1,2,xh,2,3) = {fei(a,c,f,p0,1,2,xh,2,3) }')

 meij(4,3,2,xh,2,3) = 0.026666666666666672
 keij(a,c,4,1,2,xh,2,3) = -37.32799999999999
 fei(a,c,f,p0,1,2,xh,2,3) = 4.372766997748158


In [51]:
import numpy as np
import scipy.sparse as sp

def massM(xh, shapeFn, noOfIntegPt):
    
    n_elem = len(xh) - 1  # Number of elements
    n_nodes = n_elem * shapeFn  # Total nodes based on shape function order
    
    # Initialize sparse matrix
    M = sp.lil_matrix((n_nodes, n_nodes))
    
    # Loop over each element
    for e in range(1, n_elem + 1):
        x1, x2 = xh[e-1], xh[e]  # Element endpoints
        
        # Local mass matrix for element e
        local_M = np.zeros((shapeFn + 1, shapeFn + 1))
        
        for i in range(1, shapeFn + 2):
            for j in range(1, shapeFn + 2):
                local_M[i-1, j-1] = meij(e, i, j, xh, shapeFn, noOfIntegPt)
        
        # Assemble into global matrix
        if shapeFn == 1:
            if e == 1:
                M[0,0] = local_M[-1,-1]
            else:
                start = e - 2
                for i in range(shapeFn + 1):
                    for j in range(shapeFn + 1):
                        global_i = start + i
                        global_j = start + j
                        M[global_i, global_j] += local_M[i, j]
        else:
            if e == 1:
                for i in range(1,3):
                    for j in range(1,3):
                        M[i-1,j-1] = local_M[i,j]
            else:
                for i in range(shapeFn + 1):
                    for j in range(shapeFn + 1):
                        global_i = 2*e - 3 + i        
                        global_j = 2*e - 3 + j
                        M[global_i, global_j] += local_M[i,j]
    
    return M.tocsr()  # Convert to CSR format for efficiency


In [40]:
print(f'massM(xh, 1, 2) = ')
print(f'{massM(xh, 1, 2) }')

massM(xh, 1, 2) = 
  (0, 0)	0.2666666666666667
  (0, 1)	0.06666666666666664
  (1, 0)	0.06666666666666664
  (1, 1)	0.2666666666666668
  (1, 2)	0.06666666666666665
  (2, 1)	0.06666666666666665
  (2, 2)	0.26666666666666683
  (2, 3)	0.06666666666666662
  (3, 2)	0.06666666666666662
  (3, 3)	0.2666666666666667
  (3, 4)	0.06666666666666662
  (4, 3)	0.06666666666666662
  (4, 4)	0.13333333333333336


In [52]:
xh = np.linspace(0.,1.5,4)
print(f'massM(xh, 2, 2) = ')
print(f'{massM(xh, 2, 2) }')

massM(xh, 2, 2) = 
  (0, 0)	0.2222222222222222
  (0, 1)	0.055555555555555566
  (1, 0)	0.055555555555555566
  (1, 1)	0.11111111111111108
  (1, 2)	0.05555555555555555
  (1, 3)	-0.027777777777777776
  (2, 1)	0.05555555555555555
  (2, 2)	0.22222222222222227
  (2, 3)	0.05555555555555555
  (3, 1)	-0.027777777777777776
  (3, 2)	0.05555555555555555
  (3, 3)	0.11111111111111119
  (3, 4)	0.055555555555555594
  (3, 5)	-0.027777777777777797
  (4, 3)	0.055555555555555594
  (4, 4)	0.2222222222222219
  (4, 5)	0.055555555555555594
  (5, 3)	-0.027777777777777797
  (5, 4)	0.055555555555555594
  (5, 5)	0.05555555555555566


In [6]:
import numpy as np
import scipy.sparse as sp

def stiffK(a, c, xh, shapeFn, noOfIntegPt):
    
    n_elem = len(xh) - 1  # Number of elements
    n_nodes = n_elem * shapeFn  # Total nodes based on shape function order

    # Initialize sparse matrix
    K = sp.lil_matrix((n_nodes, n_nodes))

    # Loop over each element
    for e in range(1, n_elem + 1):
        x1, x2 = xh[e-1], xh[e]  # Element endpoints
        
        # Local stiffness matrix for element e
        local_K = np.zeros((shapeFn + 1, shapeFn + 1))
        
        for i in range(1, shapeFn + 2):
            for j in range(1, shapeFn + 2):
                local_K[i-1, j-1] = keij(a, c, e, i, j, xh, shapeFn, noOfIntegPt)

        # Assemble into global matrix
        for i in range(shapeFn + 1):
            for j in range(shapeFn + 1):
                global_i = (e - 1) * shapeFn + i
                global_j = (e - 1) * shapeFn + j
                K[global_i, global_j] += local_K[i, j]

    return K.tocsr()  # Convert to CSR format for efficient calculations


In [ ]:
print(f'stiffK(a, c, xh, 1, 2) = ')
print(f'{stiffK(a, c, xh, 1, 2)}')

In [ ]:
print(f'stiffK(a, c, xh, 2, 2) = ')
print(f'{stiffK(a, c, xh, 2, 2)}')

In [7]:
import numpy as np

def loadF(a, c, f, p0, QL, xh, shapeFn, noOfIntegPt):
    
    n_elem = len(xh) - 1  # Number of elements
    n_nodes = n_elem * shapeFn + 1  # Total nodes based on shape function order

    # Initialize global load vector
    F = np.zeros(n_nodes)

    # Loop over elements
    for e in range(1, n_elem + 1):
        x1, x2 = xh[e-1], xh[e]  # Element endpoints
        
        # Local load vector
        local_F = np.zeros(shapeFn + 1)

        for i in range(1, shapeFn + 2):
            local_F[i-1] = fei(a, c, f, p0, e, i, xh, shapeFn, noOfIntegPt)

        # Assemble local contributions into global load vector
        for i in range(shapeFn + 1):
            global_i = (e - 1) * shapeFn + i
            F[global_i] += local_F[i]

    # Apply Neumann boundary condition at x = L
    F[-1] += QL

    return F


In [ ]:
print(f'loadF(a,c,f,p0,-3,xh,1, 2) = {loadF(a, c, f, p0, -3, xh, 1, 2)}')

In [ ]:
print(f'loadF(a,c,f,p0,-3,xh,2,2) = {loadF(a,c,f,p0,-3,xh,2,2)}')

In [8]:
import numpy as np

def L2norm1d(f, a, b, numElmnt):
    
    # Divide [a, b] into numElmnt subintervals
    xh = np.linspace(a, b, numElmnt + 1)

    integral_value = 0  # Initialize sum for integral

    # Loop over each element [xh[i], xh[i+1]]
    for i in range(numElmnt):
        xl, xr = xh[i], xh[i + 1]
        
        # Function to integrate (f squared)
        integrand = lambda x: f(x) ** 2
        
        # Integrate over subinterval
        integral_value += gaussQuad1d(integrand, xl, xr, 3)  # 3-point Gaussian quadrature

    return np.sqrt(integral_value)  # Compute L2 norm


In [ ]:
print(f'L2norm1d(f,0,2,5)={L2norm1d(f,0,2,5)}')

In [ ]:
f = lambda x: 3*x*np.sin(x)
print(L2norm1d(f,-2.44,1.77,9))

In [21]:
len(np.array([1, .2, .5, .8]))

4

In [39]:
for i in range(1,5):
    print(i)

1
2
3
4


In [41]:
for i in range(5):
    print(i)

0
1
2
3
4


In [10]:
xh = np.linspace(0.,2.,6)
print(xh)

[0.  0.4 0.8 1.2 1.6 2. ]


In [29]:
array_2d = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12]
])

In [34]:
array_2d[0:1,1:2]

array([[2]])